You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

In [1]:
# Import pandas
import pandas as pd

In [2]:
# Read in office_addresses.csv
offices = pd.read_csv("datasets/office_addresses.csv")
offices

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [3]:
# Declare a list of columns to keep from addresses
addresses_cols = ["employee_id", "employee_country", "employee_city", "employee_street", "employee_street_number"]

In [4]:
# Read in employee_information.xlsx
addresses = pd.read_excel("datasets/employee_information.xlsx",
                         usecols=addresses_cols)
addresses

,employee_id,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,BE,Leuven,Grote Markt,9
1,H8K0L6,GB,London,Baker Street,221
2,G4R7V0,US,New-York,Perry Street,66
3,M1Z7U9,FR,Paris,Rue de l'Université,7


In [5]:
# Declare a list of new column names
emergency_contacts_header = ["employee_id", "last_name", "first_name",
                             "emergency_contact", "emergency_contact_number", "relationship"]

In [6]:
# Read in employee_information.xlsx
emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", 
                                   sheet_name="emergency_contacts", 
                                   header=None,
                                   names=emergency_contacts_header)
emergency_contacts

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [7]:
# Read in employee_roles.json
roles = pd.read_json("datasets/employee_roles.json", orient="index")
roles

,title,monthly_salary,team
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [8]:
# Merge addresses with offices
employees = addresses.merge(offices, left_on="employee_country", right_on="office_country", how="left")
employees

,employee_id,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,FR,Paris,Rue de l'Université,7,NaN,NaN,NaN,NaN,NaN


In [9]:
# Merge employees with roles
employees = employees.merge(roles, left_on="employee_id", right_on=roles.index)

In [10]:
# Merge employees with emergency_contacts
employees = employees.merge(emergency_contacts, on="employee_id")

In [11]:
# Fill null values in office columns
for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
    employees[col].fillna("Remote", inplace=True)

In [12]:
# Create final columns
final_columns = ["employee_id", "first_name", "last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]

In [13]:
# Subset for the required columns
employees_final = employees[final_columns]
employees

,employee_id,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,FR,Paris,Rue de l'Université,7,Remote,Remote,Remote,Remote,Remote,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [14]:
# Set employee_id as the index
employees_final.set_index("employee_id", inplace=True)

In [15]:
employees_final

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
